In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt
import pandas as pd
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from Performance import performance_evaluation
perfs = performance_evaluation()

In [2]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,data_generation_process) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}; noise_train_all = {}; noise_test_all = {}
    for iter in iteration_all:
        DataPath_iter = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_iter).mkdir(parents=True, exist_ok=True)
        W_star = data_gen.generate_truth(DataPath_iter,lower, upper, p, d, iter,data_generation_process) 
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], noise_train_all[iter],noise_test_all[iter],W_star_all[iter] = data_gen.generate_samples(iter,DataPath_iter,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, noise_train_all,noise_test_all,W_star_all

In [13]:
grid = (2,2) # grid size
num_train = 10 # number of training data
num_feat = 1 # size of feature
num_test = 1000
deg = 1.0 # polynomial degree
e = 0.5 # scale of normal std or the range of uniform. For the error term

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
# d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
# num_nodes = grid[0]*grid[0]
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 100

data_generation_process = "SPO_Data_Generation"
# data_generation_process = "DDR_Data_Generation"

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)

In [14]:

middle_path = '/Data/Shortest_Path_Reproduce/Negative_Sign/'
x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; noise_train_all = {}; noise_test_all = {}; W_star_all = {}
iteration_all = np.arange(0,1)

d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
DataPath_parent = os.path.dirname(grandparent_directory) + middle_path +str(grid[0])+'by'+str(grid[1])+'_grid_' + data_generation_process + "_S=100/"
pathlib.Path(DataPath_parent).mkdir(parents=True, exist_ok=True)
print("DataPath_parent:", DataPath_parent)
DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
x_test_all[grid[0]], c_test_all[grid[0]], x_train_all[grid[0]], c_train_all[grid[0]],noise_train_all[grid[0]],noise_test_all[grid[0]],W_star_all[grid[0]] \
= Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)


DataPath_parent: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Negative_Sign/2by2_grid_SPO_Data_Generation_S=100/


In [15]:
x_train_all[2]

{np.int64(0): array([[ 0.55313206],
        [ 0.10217833],
        [ 1.17877957],
        [ 0.56144991],
        [ 1.30184623],
        [ 0.86896349],
        [-1.63880731],
        [-0.55364239],
        [ 0.02429091],
        [ 0.370825  ]])}

In [18]:
from OLS import ols_method
ols_method_obj = ols_method()
iter = 0
W_ols, w0_ols, t_ols, obj_ols = ols_method_obj.ols_solver("",x_train_all[grid[0]][0], c_train_all[grid[0]][0])

In [19]:
W_ols

[[1.124152906578105],
 [0.627457613825356],
 [1.6650356724001303],
 [0.6405684315806773]]

In [21]:
from DDR import DDR_method
ddr_object = DDR_method()

from Network import network_design
Network = network_design()
arcs,arc_index_mapping = Network._getArcs(grid)

lamb = 0.1
mu = 0.5
w0_ddr,W_ddr,alpha_rst,obj_ddr = ddr_object.solve_DDR(arcs,lamb,mu,num_nodes,x_train_all[grid[0]][0], c_train_all[grid[0]][0])


In [22]:
w0_ddr

[4.086762719809838, 4.0659676383544205, 4.404938091629465, 4.163716337051171]